# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels = 2, id2label = {0 : 'Negative', 1 : "Positive" }, label2id = {'Negative' : 0 , "Positive" : 1 })
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model.config.pad_token_id = tokenizer.pad_token_id


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [2]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes", split = "train").train_test_split(test_size = 0.2, shuffle = True)
dataset

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6824
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1706
    })
})

In [3]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize(batch):
    return tokenizer(batch['text'], padding = True, truncation = True)
train_dataset = dataset['train'].map(tokenize, batched = True)
test_dataset = dataset['test'].map(tokenize, batched = True)
#train_dataset

Map:   0%|          | 0/6824 [00:00<?, ? examples/s]

Map:   0%|          | 0/1706 [00:00<?, ? examples/s]

In [6]:
model.config.pad_token_id = model.config.eos_token_id
import torch
import numpy as np
import pandas as pd
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

for param in model.parameters():
    param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    predictions = np.argmax(predictions, axis = 1)
    return {"accuracy" : (predictions == labels).mean()}

training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 1,
    weight_decay = 0.01,
    save_strategy = "epoch",
)

In [7]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics = compute_metrics
)

In [8]:
eval_results = trainer.evaluate()
eval_results

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 8.170622825622559,
 'eval_accuracy': 0.5035169988276671,
 'eval_runtime': 9.1456,
 'eval_samples_per_second': 186.538,
 'eval_steps_per_second': 46.689}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [9]:
#PEFT Model
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels = 2, id2label = {0 : 'Negative', 1 : "Positive" }, label2id = {'Negative' : 0 , "Positive" : 1 })
for param in model.parameters():
    param.requires_grad = False


from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification
peft_config = LoraConfig(
    task_type = "SEQ_CLS",
    inference_mode = False,
    r = 4,
    lora_alpha = 32,
    lora_dropout = 0.1,
)
model.config.pad_token_id = tokenizer.pad_token_id
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 150,528 || all params: 124,590,336 || trainable%: 0.1208183594592762


In [10]:
def compute_metrics(eval_pred):
  predictions,labels = eval_pred
  predictions = np.argmax(predictions, axis = 1)
  return {"accuracy" : (predictions == labels).mean()}

In [12]:
model.config.pad_token_id = model.config.eos_token_id
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np

peft_training_args = TrainingArguments(
    output_dir = "./results/peft_model",
    evaluation_strategy = "epoch",
    logging_steps = 10,
    learning_rate = 2e-5,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 5,
    weight_decay = 0.01,
    save_strategy = "epoch",
    load_best_model_at_end = True,
    )

In [13]:
peft_trainer = Trainer(
    model = model,
    args = peft_training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics = compute_metrics
)
peft_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.578900,0.460512,0.803048
2,0.399900,0.499121,0.832356
3,0.799600,0.488316,0.844666
4,0.471800,0.483491,0.849941
5,0.392200,0.507082,0.849941


TrainOutput(global_step=8530, training_loss=0.5556017107309518, metrics={'train_runtime': 537.2757, 'train_samples_per_second': 63.506, 'train_steps_per_second': 15.876, 'total_flos': 1243403185102848.0, 'train_loss': 0.5556017107309518, 'epoch': 5.0})

In [14]:
peft_eval_results = peft_trainer.evaluate()
peft_eval_results

{'eval_loss': 0.4605118930339813,
 'eval_accuracy': 0.8030480656506448,
 'eval_runtime': 10.1647,
 'eval_samples_per_second': 167.836,
 'eval_steps_per_second': 42.008,
 'epoch': 5.0}

In [15]:
peft_model.save_pretrained("model/peft_model")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [16]:
#Performing Inference with PEFT Model
import torch
from peft import AutoPeftModelForSequenceClassification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inference_model = AutoPeftModelForSequenceClassification.from_pretrained('model/peft_model', id2label = {0 : 'Negative', 1 : "Positive" }, label2id = {'Negative' : 0 , "Positive" : 1 })


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
inference_model.config.pad_token_id = tokenizer.pad_token_id

In [20]:
inference_args = TrainingArguments(
    output_dir="./results/peft_model_inference",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end = True,
)

tuned_trainer = Trainer(
    model = peft_model,
    args = inference_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics = compute_metrics
)
tuned_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.487000,0.428019,0.832356


TrainOutput(global_step=1706, training_loss=0.47871679326154704, metrics={'train_runtime': 107.4472, 'train_samples_per_second': 63.51, 'train_steps_per_second': 15.878, 'total_flos': 248582888091648.0, 'train_loss': 0.47871679326154704, 'epoch': 1.0})

In [21]:
tuned_result = tuned_trainer.evaluate()
print("Final Result on Pretrained Model:", eval_results)
print("Final Result on LoRa Model:", peft_eval_results)
print("Final Result on Tuned Model:", tuned_result)

Final Result on Pretrained Model: {'eval_loss': 8.170622825622559, 'eval_accuracy': 0.5035169988276671, 'eval_runtime': 9.1456, 'eval_samples_per_second': 186.538, 'eval_steps_per_second': 46.689}
Final Result on LoRa Model: {'eval_loss': 0.4605118930339813, 'eval_accuracy': 0.8030480656506448, 'eval_runtime': 10.1647, 'eval_samples_per_second': 167.836, 'eval_steps_per_second': 42.008, 'epoch': 5.0}
Final Result on Tuned Model: {'eval_loss': 0.4280189573764801, 'eval_accuracy': 0.8323563892145369, 'eval_runtime': 6.7331, 'eval_samples_per_second': 253.375, 'eval_steps_per_second': 8.02, 'epoch': 1.0}
